In [1]:
import tensorflow as tf
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, df, batch_size=32, target_size=(112, 112),
                shuffle=True):
        self.len_df = len(df)
        self.batch_size = batch_size
        self.target_size = target_size
        self.shuffle = shuffle
        self.class_col = ['black', 'blue', 'brown', 'green', 'red', 'white',
                         'dress', 'shirt', 'pants', 'shorts', 'shoes']
        
        # 제네레이터를 통해서 이미지를 불러옵니다.
        self.generator = ImageDataGenerator(rescale=1./255)
        self.df_generator = self.generator.flow_from_dataframe(dataframe=df,
                                                              directory='',
                                                              x_col='image',
                                                              y_col=self.class_col,
                                                              target_size= self.target_size,
                                                              color_mode='rgb',
                                                              class_mode='other',
                                                              batch_size=self.batch_size,
                                                              seed=42)
        
        self.colors_df = df['color']
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.floor(self.len_df) / self.batch_size)
    
    # 데이터 섞습니다.
    def on_epoch_end(self):
        self.indexes = np.range(self.len_df)
        if self.shuffle:
            np.random.shuffle(self.indexes)
        
    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]
        colors = self.__data_generation(indexes)
        images, labels = self.df_generator.__getitem__(index)
        
        # 다중 입력과 출력을 반환합니다.
        return [images, colors], labels
    
    def __data_generation(self, indexes):
        colors = np.array([self.colors_df[k] for k in indexes])
        
        return colors

In [2]:
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
DATA_PATH ='C:/Users/sbs/Desktop/박진혁_수업자료/AI과정2/텐서플로우/archive'

train_df = pd.read_csv(DATA_PATH + '/train.csv')
train_datagen = DataGenerator(train_df)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/sbs/Desktop/박진혁_수업자료/AI과정2/텐서플로우/archive/train.csv'

In [13]:
# Lambda를 사용해서 모델 만들기
from tensorflow.keras.layers import Lambda, Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

# 나만의 함수를 만듭니다.
def custom_f(x):
    x_mean = K.mean(x)
    x_std = K.std(x)
    
    return (x - x_mean) / x_std

inputs = Input(shape=(5,))
#Lambda층을 통해 layer오 변환해서 연결
x = Lambda(custom_f)(inputs)
# 기존에 사용하던 층과 연결
x = Dense(32, activation='relu')(x)

# 모델 구성
model = Model(inputs=inputs, outputs=x)

import tensorflow as tf

test_list = tf.Variable([[1.,2.,3.,4.,5.]])
test_mean = K.mean(test_list)
test_std = K.std(test_list)

# 이거 해보세요
result_check = custom_f(test_list)
print("출력값 : " + str(model(test_list)))
print("결과 확인 : " + str(result_check))


The following Variables were used a Lambda layer's call (lambda_5), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 5) dtype=float32, numpy=array([[1., 2., 3., 4., 5.]], dtype=float32)>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
출력값 : tf.Tensor(
[[0.         0.0012401  0.         0.         0.7305002  0.
  0.04475333 0.53374946 0.         1.0149286  0.         0.
  0.         0.         0.         0.         0.         0.01722381
  0.23970462 0.         0.         0.         0.         0.54019356
  0.         0.         0.25671047 0.2932949  0.         0.
  1.0176364  0.02319683]], shape=(1, 32), dtype=float32)
결과 확인 : tf.Tensor([[-1.4142135  -0.70710677  0.          0.70710677  1.4142135 ]], shape=(1, 5), dtype=float32)


In [ ]:
# 커스텀 케라스층
# build(input_shape) : 학습할 가중치 정의
# call(x) : 층에서 수행할 핵심 연산을 정의
# compute_output_shape(input_shape) : 연산을 통해 입력 형태가 수정되나뎜
# 이를 명시해야 한다. 출력이 다른 층의 입력으로 사용될 때, 다음 중에서 입력의 형태를
# 추론할 수 있도록 돕는다.

In [14]:
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer

class my(Layer):
    def __init__(self, output_dim, **kwargs):
        output_dim = self.output_dim
        
        super(my, self).__init__(**kwargs)